# TorchMD Simulation

This notebook runs a short torchmd simulation of a water box.

In [1]:
#SBATCH -p gpu --exclusive --time 24:00:00
import sys
sys.path.append(".")
DEVICE = "cpu"
#DEVICE = "cuda:0"

In [2]:
import torch
import numpy as np

Set seeds to ensure reproducibility.

In [3]:
torch.manual_seed(1)
np.random.seed(1)

## Build Test System

The files defining the water box are part of the `torchmd` test suite. The setup is defined in the module `waterbox.py`.
The box contains 96 water molecules as well as one ion pair (sodium and chlorine).

In [4]:
from waterbox import WaterBox
waterbox = WaterBox(nreplicas=1, device=DEVICE)

## Energy Minimization

In [ ]:
from torchmd.minimizers import minimize_bfgs
minimize_bfgs(waterbox.system, waterbox.forces, steps=500)  # Minimize the system

Iter  Epot            fmax    
   0   -691.583104    57.640355
   1   -249.736811    212.411990
   2   -731.377728    19.837357
   3   -740.159184    20.139690
   4   -786.285572    43.255250
   5   -792.541717    66.349296
   6   -815.886302    22.912048
   7   -830.756328    21.255851
   8   -842.972508    26.154271
   9   -852.190711    18.151096
  10   -857.584258    92.193282
  11   -871.894552    28.030490
  12   -882.560370    23.991500
  13   -893.327520    46.935104
  14   -897.904242    31.250359
  15   -903.884071    12.794050
  16   -910.240079    20.790360
  17   -917.728849    23.770905
  18   -927.771507    25.981902
  19   -933.377868    28.852858
  20   -941.155280    16.347461
  21   -945.924788    15.937664
  22   -951.955510    17.664942
  23   -957.618717    48.166648
  24   -968.034143    20.388571
  25   -975.991177    19.592387
  26   -982.013636    14.397089
  27   -984.929808    30.084177
  28   -990.443738    14.777269
  29   -996.866784    15.398846
  30   -

## Prepare Simulation

In [ ]:
from torchmd.integrator import Integrator
from torchmd.wrapper import Wrapper

langevin_temperature = 300.0  # K
langevin_gamma = 10
timestep = 1  # fs

integrator = Integrator(
    waterbox.system, 
    waterbox.forces, 
    timestep, 
    waterbox.device, 
    gamma=langevin_gamma, 
    T=langevin_temperature
)
wrapper = Wrapper(
    waterbox.mol.numAtoms, 
    waterbox.mol.bonds if len(waterbox.mol.bonds) else None, 
    waterbox.device
)

In [ ]:
from torchmd.utils import LogWriter

logger = LogWriter(
    path="logs/",
    keys=('iter','ns','epot','ekin','etot','T'),
    name='monitor.csv'
)

## Equilibration

In [ ]:
*_, T = integrator.step(niter=100)
*_, T = integrator.step(niter=100)
print(T)

## Run NVE Simulation

In [ ]:
integrator = Integrator(
    waterbox.system, 
    waterbox.forces, 
    timestep, 
    waterbox.device, 
    gamma=0.0, 
    T=None
)

In [ ]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 10000
output_period = 10
save_period = 100
traj = []

system = waterbox.system
forces = waterbox.forces

trajectoryout = "xyz_vel.npy"
iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, waterbox.system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    currvel = system.vel.detach().cpu().numpy().copy()
    traj.append(np.stack([currpos[0], currvel[0]], axis=0))
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=0))

    logger.write_row({
        'iter':i*output_period,
        'ns':FS2NS*i*output_period*timestep,
        'epot':Epot,
        'ekin':Ekin,
        'etot':Epot+Ekin,
        'T':T
    })